<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 3: Improving LLMs with RLHF (DPO & GRPO)</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, October 14, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 19
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>


# 🤖 Post-Training: Improving LLMs with RLHF (DPO & GRPO)

In this notebook, we’ll show how to improve a language model using **two post-training techniques**:

### 🧠 What You’ll Learn
- What **Direct Preference Optimization (DPO)** is and how it helps models choose better answers.
- What **Group Relative Policy Optimization (GRPO)** is and how it helps models solve tasks  improving reasoning and performance on complex tasks (math, code, logic).
- How to train small models on **real feedback data**.
- How to **observe changes in model behavior** after fine-tuning.


### 📦 What We’ll Use
- **Hugging Face 🤗 Transformers** to load and run models
- **TRL (Transformer Reinforcement Learning)** library by Hugging Face 🤗 for DPO and GRPO
- **A small version** of the French translated [Anthropic HH-RLHF dataset](https://huggingface.co/datasets/AIffl/french_hh_rlhf)
- **Colab GPU**, so models are small enough to run quickly

### Useful links:
- [Hugging Face 🤗 DPO Trainer](https://huggingface.co/docs/trl/dpo_trainer)
- [Hugging Face 🤗 GRPO Trainer](https://huggingface.co/docs/trl/grpo_trainer)

> This notebook is interactive, friendly, and high-level. You don’t need to know deep math or theory to follow along.

# Quick start:
1-  Clone the repository:
```bash
git clone https://github.com/BounharAbdelaziz/RLHF.git
```
2- Install the dependencies:
```bash
pip install -q -r requirements.txt
```
Now we are ready to go!

In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/BounharAbdelaziz/RLHF.git

Cloning into 'RLHF'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 13.46 KiB | 13.46 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
!pip install -q -r RLHF/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.9 MB/s eta 0:00:00


# Part I: DPO

# 🧠 Fine-tuning Qwen2.5-0.5B-Instruct on French Data

In this part, we’ll walk through how to **fine-tune the Qwen2.5-0.5B-Instruct** model on **French-language data**, using **off-policy DPO (Direct Preference Optimization)**.

---

## 🧩 Key Concepts

- **Model**: [Qwen2.5-0.5B-Instruct](https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct)  
- **Objective**: Adapt the model for French understanding and instruction-following  
- **Method**: Off-policy **DPO** for alignment-based fine-tuning  

---

## ⚙️ System Requirements

Before training, make sure you choose an appropriate GPU setup.

- **Memory requirements**: [GPU memory guidance](https://rahulschand.github.io/gpu_poor/)  
- **GPU options**:
  - AWS SageMaker: [pricing calculator](https://aws.amazon.com/sagemaker-ai/pricing/)  
  - 💸 Cheaper alternative: [RunPod](https://www.runpod.io/)  

---

## 🧮 Memory Optimization

We’ll use **LoRA** (Low-Rank Adaptation) combined with **quantization (4-bit)** to reduce GPU memory usage while maintaining performance.

---

## 🧪 Evaluation Tools

To evaluate the fine-tuned model, you can use one or more of the following frameworks:

- [`lm-eval-harness`](https://github.com/EleutherAI/lm-evaluation-harness)
- [`lighteval`](https://github.com/huggingface/evaluate)
- [`libra-eval`](https://github.com/facebookresearch/libra)

---
Below is a visual overview of the **Direct Preference Optimization (DPO)** training process:

![DPO Training Diagram](https://1drv.ms/i/c/ae69638675180117/IQQ_wS77RKdrS4tzLbwoqr1gAR0Bf_1X2U36NRBp1Odsypg?width=560&height=48)

In [ ]:
# Imports
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from trl import (
    DPOTrainer,
    DPOConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
import torch
import os
import wandb

## 📊 Tracking with Weights & Biases (W&B)

We’ll use **Weights & Biases (W&B)** to log training metrics, model versions, and system stats so you can compare runs, debug faster, and share results. Before running the next cell, **create a free account** at [https://wandb.ai](https://wandb.ai) and make a new **Project** (e.g., `RLHF`). In the code cell that follows, we’ll initialize W&B; on first use you’ll need to **register** then paste your **API key** from your W&B profile. During training, W&B will automatically track losses, learning rate, gradient norms, and GPU utilization, and we’ll log custom metrics (e.g., validation perplexity, evaluation scores) plus configuration details (dataset, hyperparameters, LoRA/quantization settings). Each run will appear on your project dashboard with charts, tables, run metadata, and artifacts, making it easy to **compare experiments**, **resume runs**, and **share dashboards** with your teammates.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
Create your Weights&Biases account and fill the gap the next cell.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# We will use wandb.ai for logging the experiments - Set your API key here
WANDB_API_KEY = "e831d2b24ffe7f9a0c36a6fdbfefca68ae6da2c7" #API KEY memo : e831d2b24ffe7f9a0c36a6fdbfefca68ae6da2c7
# Automatically login using the API key
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
os.environ["WANDB_PROJECT"] = "RLHF"
wandb.login()

# Training dataset
DATASET_PATH = "AIffl/french_orca_dpo_pairs" # french version of "Intel/orca_dpo_pairs"

# We limit to 2k samples for speed
LIMIT = 2_000

# SFT Model we will finetune
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
# Seed for reproducibility
SEED = 1998

MAX_PROMPT_LEN = 1024
MAX_LENGTH = MAX_PROMPT_LEN + 512

RUN_NAME = "DPO-french-orca-" + MODEL_NAME.split('/')[-1]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: rubcardoso51 (rubcardoso51-cole-normale-sup-rieure-paris-saclay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Load the SFT Model and Tokenizer

We’ll stick with 4-bit quantization via bitsandbytes for this lab. You’ve already used it last week, so nothing new—same setup (load the model with 4-bit weights), same goal (reduce VRAM) with minimal impact on quality for our use case. This keeps runs feasible on a single GPU.

We will need the tonkenizer in the data preparation step to apply the chat template.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Create the quantization confiduration to load the model with 4 bits
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
quantization_config = BitsAndBytesConfig( #fill the gap
    load_in_4bit=True,
    bnb_4bit_compute_dtype = 'bfloat16',
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant =True)


# Load the model to finetune
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)
# to avoid warning
model.config.use_cache = False
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

## Data Preparation

### 💬 Chat templates & converting data to `messages`

Modern instruction-tuned models (including **Qwen2.5-0.5B-Instruct**) expect inputs in a **chat format** and rely on a **tokenizer chat template** to turn structured messages into the exact token sequence the model was trained on. In practice, you should **not** hand-craft special tokens; instead, pass a list of `{role, content}` messages to the tokenizer and let `apply_chat_template(...)` do the right thing.

#### Why a chat template?
- Ensures your prompts match the **pretraining/finetuning format** (system/user/assistant turns, BOS/EOS, separators).
- Minimizes prompt drift across libraries and models.
- Makes it easy to add **system instructions** (e.g., “You are a helpful assistant that answers in French.”).

#### Message structure
Each example becomes an ordered list of chat turns:
```python
messages = [
  {"role": "system", "content": "Tu es un assistant utile. Réponds en français."},
  {"role": "user", "content": "Explique la différence entre LoRA et le fine-tuning complet."},
  {"role": "assistant", "content": "LoRA adapte un petit sous-espace de poids, alors que..."}
]


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Create the user message which is the question field of the dataset.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def preprocess_for_dpo(example):
    # Format system message if present
    messages = []
    if example.get('system') and len(example['system'].strip()) > 0:
        messages.append({"role": "system", "content": example['system']})
    # return print(example.keys()) that was for the test to see example

    # fill the gap, add user message from the question filed of th
    user_message = {"role": "user", "content": example["question"]} # fill here
    messages.append(user_message)

    # Create prompt with generation prompt for DPO
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # The chosen and rejected should be the assistant responses only
    chosen = example['chosen']
    rejected = example['rejected']

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Download the training dataset
dataset = load_dataset(DATASET_PATH, split=f"train")
# shuffle and select a number of samples
dataset = dataset.shuffle(True).select(range(LIMIT))

# Save columns
original_columns = dataset.column_names

# Apply the preprocessing function
dpo_dataset = dataset.map(
    preprocess_for_dpo,
    remove_columns=original_columns,
)

# Filter out examples that are too long
def filter_length(example):
    prompt_length = len(tokenizer.encode(example['prompt']))
    chosen_length = len(tokenizer.encode(example['chosen']))
    rejected_length = len(tokenizer.encode(example['rejected']))

    return (prompt_length + max(chosen_length, rejected_length)) < MAX_LENGTH

dpo_dataset = dpo_dataset.filter(filter_length)

print(f"Dataset size after filtering: {len(dpo_dataset)}")

README.md:   0%|          | 0.00/468 [00:00<?, ?B/s]

french_orca_rlhf.jsonl:   0%|          | 0.00/45.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12670 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset size after filtering: 1954


## Model Training

Training will mirror last week’s lab, but we’ll switch from SFT to **off-policy DPO** using the `trl` library. Concretely, we’ll instantiate a **policy model** (trainable) and a **reference model** (frozen) and optimize with the DPO objective so the policy prefers **chosen** over **rejected** responses for the same prompt.

### What we’ll use
- **TRL**: `DPOConfig`, `DPOTrainer`
- **PEFT**: LoRA adapters on top of the base **Qwen2.5-0.5B-Instruct**
- **Quantization**: 4-bit (QLoRA-style) to fit on small GPUs
- **Logging**: W&B for metrics, configs, and artifacts

### Expected dataset columns
- `prompt` (or `messages`): the shared context (system+user turns)
- `chosen`: assistant reply preferred by annotators
- `rejected`: less-preferred reply
> If you’re keeping everything in chat format, we’ll pass lists of `{role, content}` and rely on `tokenizer.apply_chat_template(...)` inside the collator.

### Minimal training flow
1. Load tokenizer with the **chat template** and enable 4-bit loading of the base model.
2. Wrap the model with **LoRA** (target attention/MLP modules).
3. Build a `datasets.Dataset` that yields `(prompt/messages, chosen, rejected)`.
4. Define `DPOConfig` (batch size, lr, epochs, `beta`, logging/saving/eval cadence).
5. Create `DPOTrainer(policy_model, ref_model, tokenizer, train_dataset, eval_dataset, **config)`.
6. Call `trainer.train()`; optional `trainer.evaluate()` and `trainer.save_model()`.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Create the lora config with rank of 32, alpha of 64 and dropout of 0.1 on all MLP layers (execluding Embedding layers) and train the model
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e

In [ ]:
!pip install weave

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.5/717.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.7 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=27c01fb2b94ec46313abaecb5b748b3c83d9a921e977430cb8963e8a94c3ba45
  Stored in directory: /root/.cache/pip/wheels/65/c3/c3/238bf93c243597857edd94ddb0577faa

In [ ]:
import weave

# LoRA configuration - targeting the correct modules for Qwen2.5
peft_config = LoraConfig(r=32,
                         lora_alpha = 64,
                         target_modules = ["q_proj",
                                           "k_proj",
                                           "v_proj",
                                           "o_proj",
                                           "gate_proj",
                                           "up_proj",
                                           "down_proj"],
                         lora_dropout = 0.1
                         )# fill the gap A REMPLIR

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Training configuration
training_args = DPOConfig(
    beta=0.1,  # DPO temperature parameter
    learning_rate=5e-6,
    max_prompt_length=MAX_PROMPT_LEN,
    max_length=MAX_LENGTH,
    per_device_train_batch_size=1,  # Reduced for memory
    gradient_accumulation_steps=4,  # Increased to maintain effective batch size of 4 (1*4)
    num_train_epochs=1,
    max_grad_norm=1.0,
    logging_steps=1,
    save_steps=100,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",  # More memory efficient
    warmup_ratio=0.03, # 3% of the steps will be just a warmup
    save_strategy="steps",
    output_dir="./dpo_model",
    report_to="wandb",
    run_name=RUN_NAME,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=True,  # Enable mixed precision
)

# Initialize the trainer - Note: no ref_model needed when using peft_config
trainer = DPOTrainer(
    model=model,
    args=training_args,
#    peft_config=peft_config,  # This automatically handles reference model. Commented because peft_config is already an argument of the model
    processing_class=tokenizer,
    train_dataset=dpo_dataset,
)

# Print a sample to verify preprocessing
print("Sample from dataset:")
print(f"Prompt: {dpo_dataset[0]['prompt']}")
print(f"Chosen: {dpo_dataset[0]['chosen']}")
print(f"Rejected: {dpo_dataset[0]['rejected']}")

# Train
trainer.train()

trainable params: 17,596,416 || all params: 511,629,184 || trainable%: 3.4393


Extracting prompt in train dataset:   0%|          | 0/1954 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1954 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1954 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Sample from dataset:
Prompt: <|im_start|>system
Vous êtes un assistant utile, qui fournit toujours des explications. Pensez comme si vous répondiez à un enfant de cinq ans.<|im_end|>
<|im_start|>user
Prémisse:
"pour les drogues, ils appliquent la peine de mort pour ça, euh, ils sont juste très très durs et je suppose que c'est peut-être comme ça à Tokyo ou au Japon aussi, ils sont juste très durs avec les criminels" Basée sur cette prémisse, c'est l'hypothèse "Pour les drogues, ils appliquent la peine de mort, mais ils sont très très durs envers les criminels et c'est peut-être la même chose à Tokyo ou au Japon." vrai?
Choisissez votre réponse parmi :
[1]. Oui;
[2]. il n'est pas possible de le savoir ;
[3]. Non;<|im_end|>
<|im_start|>assistant

Chosen: [1]. Oui
Rejected: Ohh ! Laissez-moi y réfléchir comme un assistant super sympa et intelligent ! 🤔

Donc, vous dites que certains endroits, comme Tokyo ou le Japon, sont très durs envers les gens qui font de mauvaises choses, comme la dr

wandb: Initializing weave.
weave: Logged in as Weights & Biases user: rubcardoso51.
weave: View Weave data at https://wandb.ai/rubcardoso51-cole-normale-sup-rieure-paris-saclay/RLHF/weave
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
1,0.693100
2,0.693100
3,0.693100
4,0.693100
5,0.694900
6,0.675000
7,0.687200
8,0.693100
9,0.681100
10,0.682800


KeyboardInterrupt: 

In [ ]:
# merge LoRA adapters with the base model
save_path = "dpo_model/final_merged_dpo_model"

model = model.merge_and_unload()
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('dpo_model/final_merged_dpo_model/tokenizer_config.json',
 'dpo_model/final_merged_dpo_model/special_tokens_map.json',
 'dpo_model/final_merged_dpo_model/chat_template.jinja',
 'dpo_model/final_merged_dpo_model/vocab.json',
 'dpo_model/final_merged_dpo_model/merges.txt',
 'dpo_model/final_merged_dpo_model/added_tokens.json',
 'dpo_model/final_merged_dpo_model/tokenizer.json')

## Model Testing

We will test the DPO model via chat-app
To validate alignment gains, we’ll spin up a small **Gradio** app that queries both the **pre-DPO** model (baseline/reference) and the **post-DPO** **policy** side-by-side. The UI lets you enter a French prompt, then compares generations using the **same chat template** and decoding settings. This helps spot qualitative shifts in helpfulness, safety, and instruction-following.

In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path=save_path, #"habdine/CSC_53432_lab3_dpo",
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (DPO vs Base) for French",
    DPO_TEST=True,
    FRENCH_TEST=True,
)

🔄 Loading DPO Model from: dpo_model/final_merged_dpo_model
✅ DPO Model loaded successfully!
🔄 Loading Base Model from: Qwen/Qwen2.5-0.5B-Instruct
✅ Base Model loaded successfully!


/content/RLHF/chat_app.py:174: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_dpo_ui = gr.Chatbot(
/content/RLHF/chat_app.py:190: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_base_ui = gr.Chatbot(


🚀 Launching Dual-Model Gradio Chat Interface…
🟢 Finetuned model: dpo_model/final_merged_dpo_model
🔵 Base model: Qwen/Qwen2.5-0.5B-Instruct
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5e9d5cca4397626a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path="BounharAbdelaziz/Qwen2.5-0.5B-DPO-English-Orca",
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (DPO vs Base) for English",
    DPO_TEST=True,
    FRENCH_TEST=False,
)

🔄 Loading DPO Model from: BounharAbdelaziz/Qwen2.5-0.5B-DPO-English-Orca


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

✅ DPO Model loaded successfully!
🔄 Loading Base Model from: Qwen/Qwen2.5-0.5B-Instruct
✅ Base Model loaded successfully!
🚀 Launching Dual-Model Gradio Chat Interface…

/content/RLHF/chat_app.py:174: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_dpo_ui = gr.Chatbot(
/content/RLHF/chat_app.py:190: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_base_ui = gr.Chatbot(



🟢 Finetuned model: BounharAbdelaziz/Qwen2.5-0.5B-DPO-English-Orca
🔵 Base model: Qwen/Qwen2.5-0.5B-Instruct
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0662000fc7e31ef47e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Part II: GRPO


The following diagram illustrates the **GRPO (Generative Reinforcement Preference Optimization)** process — an alternative to DPO that directly optimizes generation quality from preference data using reinforcement-style updates:

![GRPO Training Overview](https://1drv.ms/i/c/ae69638675180117/IQQ-KizPdUxCRZU9qDGcpX1AAeettH1uhsJqqM1WjXiYR6s?width=705&height=66)

After exploring DPO, we now move on to **GRPO** — a reinforcement learning–style approach that builds directly on preference data.  
While **DPO** adjusts the model using an *analytic loss* derived from preference pairs, **GRPO** takes a more dynamic route: it uses **reward modeling and policy gradients** to optimize the model through sampled generations.

In essence:
- GRPO **learns from human (or model) preferences** using *on-policy* updates.  
- It combines elements of **PPO** (Proximal Policy Optimization) with **preference-based rewards** rather than explicit numerical scores.  
- This allows the model to better capture *generation quality* aspects that aren’t directly expressible through static loss terms.

In the next section, we’ll explore how to configure and launch a **GRPO training loop** using `trl`, reusing much of our previous setup (tokenizer, LoRA, quantization) but switching to **on-policy optimization**.

In [ ]:
import torch
import re
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training,
)
from trl import (
    GRPOConfig,
    GRPOTrainer,
)
import os
import wandb

# We will use wandb.ai for logging the experiments - Set your API key here
WANDB_API_KEY = "e831d2b24ffe7f9a0c36a6fdbfefca68ae6da2c7" # fill the gap with your wandb key

# Automatically login using the API key
os.environ["WANDB_API_KEY"] = WANDB_API_KEY
os.environ["WANDB_PROJECT"] = "RLHF"
wandb.login()

# Training dataset
DATASET_PATH = "openai/gsm8k"

# We limit to 200 samples for speed
LIMIT = 200

# SFT Model we will finetune
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
# Seed for reproducibility
SEED = 1998

USE_LORA = True

if MODEL_NAME == "Qwen/Qwen2.5-0.5B-Instruct":
  USE_QUANT = False
else:
  USE_QUANT = True

lora_alpha = 128
lora_r = 64
lora_dropout = 0.1

if not USE_LORA:
  MAX_PROMPT_LEN = 512
  MAX_LENGTH = MAX_PROMPT_LEN + 512
else:
    MAX_PROMPT_LEN = 150
    MAX_LENGTH = MAX_PROMPT_LEN + 150

RUN_NAME = "GRPO-GSM8K-limit-" + str(LIMIT) + "-" + MODEL_NAME.split('/')[-1]

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## Load the SFT Model and Tokenizer

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model to finetune
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    quantization_config=quantization_config if USE_QUANT else None,
    device_map="auto",
    trust_remote_code=True,
)

# Prepare model for k-bit training
if USE_QUANT:
  model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

# Add padding token if not exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if USE_LORA:

  # Configure LoRA
  lora_config = LoraConfig(
      r=lora_r,  # Rank of adaptation
      lora_alpha=lora_alpha,  # LoRA scaling parameter
      target_modules=[
          "q_proj",
          "k_proj",
          "v_proj",
          "o_proj",
          "gate_proj",
          "up_proj",
          "down_proj",
      ],  # Target modules for Qwen2.5 architecture
      lora_dropout=lora_dropout,  # LoRA dropout
      bias="none",  # Bias type
      task_type=TaskType.CAUSAL_LM,  # Task type
  )

  # Apply LoRA to the model
  model = get_peft_model(model, lora_config)

  # Print trainable parameters
  model.print_trainable_parameters()

trainable params: 35,192,832 || all params: 529,225,600 || trainable%: 6.6499


## Data Preparation

For GRPO training, we’ll use the **GSM8K** dataset — a benchmark of grade-school math word problems.  
Each problem includes a **question** and a **final answer**. Our goal is to teach the model to reason in French (or English if you prefer) and **output only the final numeric answer** enclosed between `<answer>` and `</answer>` tags.

This format makes automatic evaluation trivial — we can extract the number between tags and compare it directly to the reference.

With GRPO the model will learn two main things:
- How follow the instruction to output the correct format.
- Gain more math capabilities

---

### 🧠 Why structure as chat messages?

Just like with DPO, the **Qwen2.5-0.5B-Instruct** model expects inputs in a *chat-style message format*.  
We’ll use:
- A **system message** to define the task and output style.
- A **user message** with the math problem.
- An **assistant message** containing the reasoning and final numeric answer wrapped in tags.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Create your training samples as following:<br>
1- system message: R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS <br>
2- one-shot example of user message of "What is 2+2?" and an assistant message of "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n$<answer>$4$</answer>$" <br>
3- the question sample from the dataset as a user message.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
# Load GSM8K dataset
if LIMIT:
    dataset = load_dataset(DATASET_PATH, "main", split=f"train[:{LIMIT}]")  # Small subset for demo
else:
    dataset = load_dataset(DATASET_PATH, "main")

R1_STYLE_SYSTEM_PROMPT = """A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
The assistant first thinks shortly about the reasoning process in the mind and then provides the user with the answer in a new line between <answer> and </answer>."""

TASK_SPECIFIC_INSTRUCTIONS = "The answer must be a single integer."

def preprocess_dataset(dataset, chunk_size=1000) -> Dataset:

    def extract_hash_answer(text: str) -> str | None:
        try:
            return text.split("####")[1].strip()
        except IndexError:
            return None

    def process_batch(batch):
      prompts = []
      for question in batch["question"]:
          prompt = [
              {"role": "system", "content": R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS},
              {"role": "user", "content": "What is 2+2?"},
              {"role": "assistant", "content": "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n<answer>4</answer>"},
              {"role": "user", "content": question}
          ]
          prompts.append(prompt)
      return {
            'prompt': prompts,
            'answer': [extract_hash_answer(a) for a in batch['answer']]
        }

    return dataset.map(process_batch, batched=True, batch_size=chunk_size)
train_dataset = preprocess_dataset(dataset, chunk_size=500)

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': '72',
 'prompt': [{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it.\nThe assistant first thinks shortly about the reasoning process in the mind and then provides the user with the answer in a new line between <answer> and </answer>.\nThe answer must be a single integer.',
   'role': 'system'},
  {'content': 'What is 2+2?', 'role': 'user'},
  {'content': 'To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n<answer>4</answer>',
   'role': 'assistant'},
  {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
   'role': 'user'}]}

## Reward Function Design

We’ll use **two simple rewards** during GRPO rollouts:

1. **Format reward** — checks that the **last non-empty line** is exactly in the form  
   `<answer>NUMBER</answer>`  
   - Score: **1** if correct format, **0** otherwise.

2. **Correctness reward** — checks whether the extracted number matches the gold answer.  
   - Score: **2** if correct, **0** otherwise.

Total reward per sample ∈ {0, 1, 2, 3}.



<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
write the `extract_xml_answer` function to extract the answer from the generated text.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [ ]:
def extract_xml_answer(text: str) -> str:
    match = re.search(r"^(?:[^\r\n]*\r?\n)+<answer>\s*(\d+)\s*</answer>\r?\n?$",text)
    if match:
      return match.group(1)
    return None

def format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has the correct format."""
    pattern = r"^(?:[^\r\n]*\r?\n)+<answer>\d+</answer>\r?\n?$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [bool(re.match(pattern, r)) for r in responses]
    return [1.0 if match else 0.0 for match in matches]

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """Reward function that checks if the answer is correct."""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

In [ ]:
import os
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"  # force accelerate à ne pas activer AMP


## Model Training

In [ ]:
# GRPO Configuration
grpo_config = GRPOConfig(
    output_dir="./grpo_model",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    max_prompt_length=MAX_PROMPT_LEN,
    max_completion_length=MAX_LENGTH,
    num_generations=2, # The effective train batch size must be evenly divisible by the number of generations per prompt
    beta=0,
    epsilon=0.28,
    temperature=1,
    logging_steps=1,
    save_steps=25,
    save_total_limit=3,
    # load_best_model_at_end=True,
    # metric_for_best_model="reward",
    # greater_is_better=True,
    run_name=RUN_NAME,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit" ,  # More memory efficient
    warmup_ratio=0.03, # 3% of the steps will be just a warmup
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=False,
    bf16=False# Enable mixed precision
)
del trainer
import torch
torch.cuda.empty_cache()

# Initialize trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[format_reward_func, correctness_reward_func],
    args=grpo_config,
    train_dataset=train_dataset,
    processing_class=tokenizer,
)

# Training
print("Starting GRPO training...")
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting GRPO training...


Step,Training Loss
1,0.000000
2,0.067500
3,0.238600
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,-0.023300
9,0.000000
10,-0.014700


KeyboardInterrupt: 

In [ ]:
save_path = "grpo_model/final_merged_grpo_model"

m = trainer.model
try:
    # Cas LoRA : on fusionne les adapters dans les poids
    merged = m.merge_and_unload()
    merged.save_pretrained(save_path)
except AttributeError:
    # Pas de LoRA (ou déjà fusionné) : on sauvegarde tel quel
    m.save_pretrained(save_path)

tokenizer.save_pretrained(save_path)
print("Saved to:", save_path)


Saved to: grpo_model/final_merged_grpo_model


## Model Testing

In [ ]:
from RLHF.chat_app import launch_chat_app

launch_chat_app(
    model_path=save_path,#"habdine/CSC_53432_lab3_grpo"
    base_model_path="Qwen/Qwen2.5-0.5B-Instruct",
    title="🤖 Dual-Model Qwen Chat (GRPO vs Base) for Math",
    DPO_TEST=False,
)

🔄 Loading DPO Model from: grpo_model/final_merged_grpo_model
✅ DPO Model loaded successfully!
🔄 Loading Base Model from: Qwen/Qwen2.5-0.5B-Instruct
✅ Base Model loaded successfully!


/content/RLHF/chat_app.py:174: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_dpo_ui = gr.Chatbot(
/content/RLHF/chat_app.py:190: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_base_ui = gr.Chatbot(


🚀 Launching Dual-Model Gradio Chat Interface…
🟢 Finetuned model: grpo_model/final_merged_grpo_model
🔵 Base model: Qwen/Qwen2.5-0.5B-Instruct
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20f77ab04e64cabe00.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 📈 Model Evaluation

Once our GRPO-trained model is ready, we need to **evaluate its performance**  — to verify that it has learned to produce correctly formatted and accurate answers.

For computational issues, we will evaluate on the first 200 samples only.


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
import numpy as np
from typing import List, Dict
import json
from datetime import datetime
import logging


def extract_hash_answer(text: str) -> str | None:
    try:
        return text.split("####")[1].strip()
    except IndexError:
        return None

def evaluate_model(
    model_path: str,
    batch_size: int = 1,
    num_samples: int = None,
    save_results: bool = True,
) -> Dict:
    print("Initializing evaluation...")

    with tqdm(total=2, desc="Loading model components") as pbar:
        llm = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="cuda:0",
            trust_remote_code=True,
        )
        pbar.update(1)

        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            model_max_length=768,
        )
        pbar.update(1)

    # Load test dataset
    print("Loading dataset...")
    dataset = load_dataset('openai/gsm8k', 'main', split='test')
    if num_samples:
        dataset = dataset.select(range(num_samples))
    total_samples = len(dataset)
    print(f"Loaded {total_samples} samples")

    results = []
    correct = 0
    total = 0

    # Create progress bar
    progress_bar = tqdm(
        total=total_samples,
        desc="Processing samples",
        unit="examples",
        dynamic_ncols=True,
    )

    progress_bar.set_postfix({
        'acc': '0.00%',
        'correct': '0',
    })

    # Process in batches
    for i in range(0, total_samples, batch_size):
        batch_data = dataset[i:i + batch_size]
        current_batch_size = len(batch_data['question'])

        # Prepare prompts using same format as training
        prompts = [
            [
                {'role': 'system', 'content': R1_STYLE_SYSTEM_PROMPT + "\n" + TASK_SPECIFIC_INSTRUCTIONS},
                {'role': 'user', 'content': "What is 2+2?"},
                {'role': 'assistant', 'content': "To calculate 2+2, we simply add the numbers together: 2 + 2 = 4.\n<answer>4</answer>"},
                {'role': 'user', 'content': q.strip()}
            ] for q in batch_data['question']
        ]

        # Convert to chat format
        formatted_prompts = [
            tokenizer.apply_chat_template(
                p,
                tokenize=True,
                return_tensors='pt',
                add_generation_prompt=True
            )
            for p in prompts
        ]

        # Generate responses
        outputs = []
        for prompt in formatted_prompts:
            output = llm.generate(
                prompt.to('cuda:0'),
                max_new_tokens=512,
                temperature=1.0,
            )
            outputs.append(output)


        # Process responses
        for j, output in enumerate(outputs):

            response = tokenizer.decode(output[0], skip_special_tokens=True)

            # Extract answers
            generated_answer = extract_xml_answer(response)
            true_answer = extract_hash_answer(batch_data['answer'][j])

            # Store result
            result = {
                'question': batch_data['question'][j],
                'true_answer': true_answer,
                'generated_answer': generated_answer,
                'full_response': response,
                'correct': generated_answer == true_answer
            }
            results.append(result)

            # Update metrics
            if generated_answer == true_answer:
                correct += 1
            total += 1

        # Update progress
        progress_bar.update(current_batch_size)
        progress_bar.set_postfix({
            'acc': f'{(correct/total)*100:.2f}%',
            'correct': f'{correct}/{total}',
        })

    progress_bar.close()

    # Calculate metrics
    accuracy = correct / total if total > 0 else 0
    metrics = {
        'accuracy': accuracy,
        'correct': correct,
        'total': total,
        'model_path': model_path,
        'timestamp': datetime.now().isoformat()
    }

    # Save results
    if save_results:
        save_path = f"gsm8k_eval_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(save_path, 'w') as f:
            json.dump({
                'metrics': metrics,
                'results': results
            }, f, indent=2)
        print(f"\nResults saved to {save_path}")

    return metrics

print("Starting GSM8K evaluation...")


Starting GSM8K evaluation...


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 7: </b><br>
evaluate the model before and after GRPO training.
<hr style="border:10px solid blue"> </hr>
</font></h4>

### Evaluation after GRPO



In [ ]:
evaluate_model(
    model_path=save_path,
    batch_size=1,
    num_samples=200,
    save_results=True,
)

Initializing evaluation...


Loading model components:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset...
Loaded 200 samples


Processing samples:   0%|          | 0/200 [00:00<?, ?examples/s]


Results saved to gsm8k_eval_results_20251019_190547.json


{'accuracy': 0.195,
 'correct': 39,
 'total': 200,
 'model_path': 'grpo_model/final_merged_grpo_model',
 'timestamp': '2025-10-19T19:05:47.853451'}

### Evaluation before GRPO

In [ ]:
# with DPO
evaluate_model(
    model_path="dpo_model/final_merged_dpo_model",
    batch_size=1,
    num_samples=200,
    save_results=True,
)

Initializing evaluation...


Loading model components:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset...
Loaded 200 samples


Processing samples:   0%|          | 0/200 [00:00<?, ?examples/s]

KeyboardInterrupt: 

The accuracy before GRPO is around 3% (I could not complete the full training due to limited GPU resources), whereas after GRPO it becomes nearly seven times higher. This suggests that the GRPO fine-tuning method effectively improves accuracy. However, the final accuracy remains relatively low, which indicates that further research into more advanced fine-tuning methods is still necessary.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1: </b><br>

- What are the main conceptual and practical differences between **Direct Preference Optimization (DPO)** and **Generative Reinforcement Preference Optimization (GRPO)**?  
- In what situations might one method outperform the other?  
- How do both approaches approximate the **KL-regularized RL objective** derived in [Christiano et al., 2017](https://arxiv.org/abs/1706.03741)?

📖 *References:*  
- Rafailov et al., 2023. *Direct Preference Optimization: Your Language Model is Secretly a Reward Model.* [arXiv:2305.18290](https://arxiv.org/abs/2305.18290)
<hr style="border:10px solid red"> </hr>
</font></h4>  


<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 1: </b><br>

I/
DPO reframes preference learning as a direct comparison of model logits, without using an explicit reward model or a policy-gradient update. It simply increases the likelihood of a preferred response $y^+$ over a rejected one $y^-$, using a closed-form objective based on log-ratios:
$
\Delta_{\text{DPO}} = \log \pi_\theta(y^+|x) - \log \pi_\theta(y^-|x)
$
In contrast, Generative Reinforcement Preference Optimization (GRPO) follows a reinforcement learning paradigm, where the policy is updated using reward-based gradients:
$
\nabla_\theta J(\pi_\theta) \propto \mathbb{E}_{\pi_\theta} \left[ \nabla_\theta \log \pi_\theta(y|x) \cdot r(y) \right]
$
Conceptually, DPO removes the need for sampling trajectories and reward estimation, making it simpler and more stable, while GRPO retains exploration and can adapt beyond the dataset but requires KL control and variance reduction.

II/ DPO is advantageous in supervised alignment settings, especially when working with static preference datasets and limited compute. It is well-suited for refining style, helpfulness, or safety without introducing instability. GRPO may outperform DPO in scenarios that require exploration, long-horizon reasoning, or interactive feedback, where a reward-based policy can discover new strategies beyond the chosen/rejected pairs.

III/

Both approaches aim to optimize a policy under a KL constraint with respect to a reference model, as defined in:
$
\max_{\pi} \; \mathbb{E}_{\pi} \left[ R(x, y) - \beta \, \mathrm{KL}\left( \pi(y|x) \,\|\, \pi_0(y|x) \right) \right]
$
GRPO enforces this objective explicitly by adding a KL penalty to the reward during policy updates. DPO embeds the KL constraint implicitly within its closed-form preference objective:
$
L_{\text{DPO}} = - \mathbb{E} \left[
\log \sigma \left(
\beta \left(
\log \frac{\pi_\theta(y^+|x)}{\pi_{\text{ref}}(y^+|x)} -
\log \frac{\pi_\theta(y^-|x)}{\pi_{\text{ref}}(y^-|x)}
\right)
\right)
\right]
$
Thus, while GRPO relies on explicit reward modeling and policy gradients, and DPO relies on a direct preference objective, both methods approximate the same KL-regularized principle behind RLHF.



<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: </b><br>

- Why can **reward model overfitting** or **reward hacking** occur in reinforcement fine-tuning?  
- How do DPO and GRPO attempt to mitigate this without explicit online reward models?  
- Discuss the role of the **reference model** in maintaining stability.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 2: </b><br>

Reward model overfitting or reward hacking occurs when a model learns to exploit weaknesses in a reward signal rather than improving true alignment or reasoning quality. Instead of generating genuinely correct or helpful outputs, the model discovers “shortcuts” that score highly under the reward setup, which can lead to unnatural or low-quality responses.

DPO and GRPO attempt to mitigate this problem without relying on an explicit online reward model.

DPO does not train a separate reward network. Instead, it optimizes the policy directly from human preference pairs (chosen vs. rejected responses). By comparing relative preferences instead of absolute reward scores, DPO avoids the risk of the model overfitting to a numerical reward and stays closer to human intent.

GRPO uses on-policy sampling with preference-based updates, but still avoids a static learned reward model. It computes advantages from generated responses and human preferences, reducing reliance on a fixed reward function. Since GRPO updates using fresh rollouts, it is less vulnerable to exploiting a frozen reward model.

The reference model plays a critical role in stability for both methods. It acts as an anchor that limits how far the new policy can move away from the original pretrained model. Through an implicit or explicit KL constraint, the policy is discouraged from deviating too much, preventing reward hacking and preserving fluency and safety. In short, the reference model maintains alignment by constraining excessive optimization.

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: </b><br>

- In the GSM8K setup, we used a simple binary reward for **format** and **correctness**.  
  - What are the limitations of such sparse rewards?  
  - How could you design a **richer, smoother** reward signal for math reasoning?
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 3: </b><br>

In the GSM8K setup, using a binary reward (1 for correct, 0 for incorrect) has important limitations:

It provides very sparse feedback: the model learns nothing when it is almost correct.

It treats all mistakes equally, even a small numerical error is punished the same as a completely wrong answer.

It does not reward good reasoning steps, only the final answer.

To design a richer and smoother reward signal, we could use:

- A partial score based on numerical proximity to the correct answer.

- Additional reward for showing correct reasoning steps or valid intermediate calculations.

- A penalty or bonus based on clarity and conciseness.

- A weighted combination of these components would give the model continuous feedback, helping it improve reasoning and not just the final output.

I found that some newer methods go beyond DPO and GRPO by providing more stable preference optimization. One example is Step-DPO (or Iterative Preference Optimization), which optimizes the model at the reasoning-step level rather than only on final answers.
The step-wise DPO objective can be written as:

$
L_{\text{Step-DPO}}
= - \mathbb{E}_{(x, y^+, y^-)}
\left[
\sum_{t=1}^{T}
\log \sigma
\left(
\beta
\left(
\log \pi_{\theta}(y^{+}_{t} \mid x, y^{+}_{<t})
-
\log \pi_{\theta}(y^{-}_{t} \mid x, y^{-}_{<t})
\right)
\right)
\right]
$

(Reference : Xiong et al., 2023
“Iterative Preference Learning from Human Feedback: Bridging Theory and Practice for RLHF under KL-Constraint.”).
<hr style="border:10px solid red"> </hr>
</font></h4>